In [22]:
import numpy as np
jnp = np
# 65535 select left
#select_upper = 65535 #jnp.packbits(jnp.arange(32) < 16).view(jnp.int32)
# print(select_upper.dtype)
def pack(val, index):
  select_upper = 65535
  # val and index both 16 bit dtypes bitcast to i32
  l = ((val << 16) & ~select_upper) | (index & select_upper)
  r = (val & ~select_upper) | ((index >> 16) & select_upper)
  return (l, r)

def unpack(l, r):
  select_upper = 65535
  vals, index = (
   ((l >> 16) & select_upper) | (r & ~select_upper),
   (l & select_upper) | ((r << 16) & ~select_upper)
  )
  return (vals, index)

'''
l, r = map(jnp.sort, pack(v.view(jnp.int32),i.view(jnp.int32)))
print(v,i)
vp, ip = unpack(l,r)
print(vp.view(v.dtype)[::2], vp.view(v.dtype)[1::2], ip.view(i.dtype))
'''
### start

import functools
import math

import jax
import jax.numpy as jnp
from jax.experimental import pallas as pl
from jax.experimental.pallas import tpu as pltpu

select_upper = 65535 #jnp.array(65535, dtype=jnp.int32) # jnp.packbits(jnp.arange(32) < 16).view(jnp.int32)

def _pack(val, index):
  # val and index both 16 bit dtypes bitcast to i32
  l = ((val << 16) & ~select_upper) | (index & select_upper)
  r = (val & ~select_upper) | ((index >> 16) & select_upper)
  return (l, r)

def _unpack(l, r):
  vals, index = (
   ((l >> 16) & select_upper) | (r & ~select_upper),
   (l & select_upper) | ((r << 16) & ~select_upper)
  )
  return (vals, index)

def blockwise_topk_packed(
 logits,
 k,
 block_topk_val=None,
 block_topk_index=None,
 start_k=0,
 num_blocks=128,
 mode='jax',
):
 '''Compute blockwise top-k.'''
 ntokens = logits.shape[0]
 assert (logits.dtype==jnp.int32) and (pl.cdiv(logits.shape[1], num_blocks) < jnp.iinfo(jnp.uint16).max)

 # pack
 # vals are in bf16, indexs in uint16 but both bitcast to int32
 block_topk_packed = []
 for i in range(k):
   block_topk_packed.append(
     list(_pack(block_topk_val[i], block_topk_index[i]))
   )

 def while_body(block_i, while_carry):
   block_topk_packeds = while_carry

   if mode == 'pallas':
     val_carry = logits[..., pl.dslice(num_blocks * block_i, num_blocks)]
     print(val_carry.shape, logits.shape, val_carry.dtype, logits.dtype)
   else:
     raise ValueError(
       'mode must be either `pallas` and a memory ref or `jax` and an array'
     )
   # block_i in int32 is the same as uint16 value padded to 32 bits
   index_carry = block_i #jnp.array(block_i, dtype=jnp.int32)

   bubbles = (
     ((val_carry << 16) & ~select_upper) | index_carry,
     (val_carry & ~select_upper) | index_carry
   )

   for i, bubble in enumerate(bubbles):
     for j in range(k):
       if j < start_k:
         # Nothing will be exchanged into the completed block topk, we just need
         # to invalidate it from flowing downward. So we check if it's already
         # found and invalidate if so.
         bubble = jnp.where(
           bubble == block_topk_packed[j][i], jnp.iinfo(jnp.int32).min, bubble
         )
       else:
         # Sinking bubble sort
         mask = bubble > block_topk_packed[j][i]
         block_topk_packed[j][i], bubble = (
           jnp.where(v, bubble , block_topk_packed[j][i]) for v in (mask, ~mask)
         )
   return block_topk_packed
 block_topk_packed = jax.lax.fori_loop(0, logits.shape[-1] // num_blocks, while_body, block_topk_packed)
 # unpack
 for i in range(start_k, k):
    block_topk_val[i], block_topk_index[i] = _unpack(*block_topk_packed[i])
 return block_topk_val, block_topk_index


# Pallas kernel scaffolding
def topk_blockwise_superset_kernel_packed(
 logits_ref,
 block_topm_val_refs,
 block_topm_index_refs,
 max_m_ref,
 flag_ref,
 num_blocks: int = 128,
 k: int = 64,
 m_schedule: tuple[int] | None = None,
):
 '''Compute blockwise top-m's until they contain global top-k.'''
 ### Initialize refs
 assert logits_ref.dtype == jnp.bfloat16
 shape = block_topm_val_refs[0].shape
 for i in range(len(block_topm_val_refs)):
   block_topm_val_refs[i][...] = jnp.full(shape, float('-inf'), dtype=logits_ref.dtype)
   block_topm_index_refs[i][...] = jnp.full(shape, jnp.iinfo(jnp.uint16).max, dtype=jnp.uint16)

 block_token = logits_ref.shape[0]
 for i in range(block_token):
   # Worst case m = k
   max_m_ref[pl.program_id(0) * block_token + i] = k

 # flag for termination of while loop
 flag_ref[0] = 0

 ### Run increasing block topk, until sure overall topk present
 if m_schedule is None:
   m_schedule = (5, 8, 12)
 # Ensure worst case of all k in one block is covered
 m_schedule = (0,) + m_schedule + (k,)

 for completed_m, m in zip(m_schedule, m_schedule[1:]):

   @pl.when(flag_ref[0] == 0)
   def _():
     topk_vals, topk_indexs = blockwise_topk_packed(
       logits_ref.bitcast(jnp.int32),
       block_topk_val=jax.tree.map(
         lambda ref: ref.bitcast(jnp.int32)[...], block_topm_val_refs
       ),
       block_topk_index=jax.tree.map(lambda ref: ref.bitcast(jnp.int32)[...], block_topm_index_refs),
       k=m,
       start_k=completed_m,
       mode='pallas',
     )

     for i in range(completed_m, m):
       block_topm_val_refs[i].bitcast(jnp.int32)[...] = topk_vals[i]
       block_topm_index_refs[i].bitcast(jnp.int32)[...] = topk_indexs[i]

     # Stopping criterion check
     # To find top-k values of a set, we can split into N subsets,
     # and sort the largest, 2nd-largest, 3-rd largest, ..., m-th largest values for each subset
     # When in the superset of top-(m-1) subsets there are more than k values
     # larger (or equal than) the largest m'th largest value from the subsets
     # then the top-(m-1) subsets must contain the top-k of the set.
     # We run a schedule of m's until we have that full top-k found.
     pivot_point = block_topm_val_refs[m - 1][...].astype(jnp.float32).max(-1, keepdims=True)
     n_larger = (
       sum([(v[...].astype(jnp.float32) >= pivot_point) for v in block_topm_val_refs[: m - 1]])
       .astype(jnp.float32)
       .sum(-1)
     )
     # flag SMEM used to check if all searches terminated
     flag_ref[0] = 0
     for i in range(block_token):
       blockwise_topm_contains_topk = n_larger[i] >= k
       flag_ref[0] += blockwise_topm_contains_topk
       # Store when the criteria was hit for each query
       token_index = pl.program_id(0) * block_token + i
       max_m = max_m_ref[token_index]
       max_m_ref[token_index] = jnp.where(
         blockwise_topm_contains_topk & (max_m == k), m - 1, max_m
       )

     # If not all terminated, reset the flag say we need to search deeper
     @pl.when(flag_ref[0] != block_token)
     def _():
       flag_ref[0] = 0


# Pallas function
def topk_blockwise_superset_pallas(
 logits, k, num_blocks=128, block_token=None, m_schedule=None
):
 num_tokens, vocab_size = logits.shape
 if block_token is None:
   block_token = min(32, num_tokens)
 if num_tokens % block_token != 0:
   raise ValueError('token block size must be a multiple of num tokens')

 out_shape = (
   [jax.ShapeDtypeStruct((num_tokens, num_blocks), logits.dtype) for i in range(k)],
   # uint16 fits vocab size of up to 2**16 * 128 = 8.4M. But not used to avoid unforseen issues.
   [jax.ShapeDtypeStruct((num_tokens, num_blocks), jnp.uint16) for i in range(k)],
   jax.ShapeDtypeStruct((num_tokens,), jnp.int32),
   jax.ShapeDtypeStruct((1,), jnp.int32),  # scratch for termination flag
 )
 out_specs = jax.tree.map(
   lambda _: pl.BlockSpec((block_token, num_blocks), lambda i: (i, 0)), out_shape[:2]
 )
 out_specs += (
   pl.BlockSpec(memory_space=pltpu.SMEM),
   pl.BlockSpec(memory_space=pltpu.SMEM),
 )
 return pl.pallas_call(
   functools.partial(
     topk_blockwise_superset_kernel_packed,
     k=k,
     num_blocks=num_blocks,
     m_schedule=m_schedule,
   ),
   in_specs=(pl.BlockSpec((block_token, vocab_size), lambda i: (i, 0)),),
   out_shape=out_shape,
   grid=(num_tokens // block_token),
   out_specs=out_specs,
   compiler_params=pltpu.TPUCompilerParams(vmem_limit_bytes=2**26),
 )(logits)


def topk_on_filtered_subset(block_topm_val, block_topm_index, k):
 num_blocks = block_topm_val[0].shape[-1]
 topk_logits, local_indices = jax.lax.top_k(
   jnp.concatenate(block_topm_val, axis=-1), k=k
 )

 @jax.vmap
 def unravel_indices(local_indices, block_topm_index):
   # TODO: consider rewriting as lane permutations
   m, col = jnp.unravel_index(local_indices, (k, num_blocks))
   row = jnp.stack(block_topm_index)[m, col]
   flat_index = row.astype(jnp.int32) * num_blocks + col.astype(jnp.int32)
   return flat_index

 topk_flat_indices = unravel_indices(local_indices, block_topm_index)
 return topk_logits, topk_flat_indices


@functools.partial(
 jax.jit,
 static_argnames=(
   'k',
   'num_blocks',
   'm_stage1_schedule',
   'm_stage2_schedule',
   'block_token',
 ),
)
def topk_optimized(
 logits,
 k: int = 64,
 num_blocks: int = 128,
 m_stage1_schedule: tuple[int] | None = None,
 m_stage2_schedule: tuple[int] | None = None,
 block_token: int | None = None,
):
 '''Fast implementation of jax.lax.top_k on TPUs.'''
 if logits.ndim != 2:
   raise ValueError('Expected 2D input')
 block_topm_val, block_topm_index, termination_m, _ = topk_blockwise_superset_pallas(
   logits,
   k=k,
   block_token=block_token,
   m_schedule=m_stage1_schedule,
   num_blocks=num_blocks,
 )
 jax.debug.print('{} {} {}', termination_m, block_topm_val[::64], block_topm_index[::64])

 # top-k the smallest number of values we can, by taking max m required
 # such that all queries to have full top-k
 # We compile for a range of shapes, then use jax.lax.cond to run just one.
 # in practice 8 nearly always sufficient
 if m_stage2_schedule is None:
   m_init = 8
   m_stage2_schedule = [
     m_init * (2**i) for i in range(int(math.log2(k // m_init)) + 1)
   ]
 # Guarantee all cases covered
 m_stage2_schedule = (-1,) + tuple(m_stage2_schedule) + (k,)

 # Buffer for output to be written in to
 topk_logits, topk_flat_indices = jax.tree.map(
   jnp.zeros_like,
   topk_on_filtered_subset(block_topm_val[:1], block_topm_index[:1], k=k),
 )
 max_m = termination_m.max()
 for lower_m, upper_m in zip(m_stage2_schedule, m_stage2_schedule[1:]):
   topk_logits, topk_flat_indices = jax.lax.cond(
     (max_m > lower_m) & (max_m <= upper_m),
     lambda *args: topk_on_filtered_subset(
       block_topm_val=block_topm_val[:upper_m],
       block_topm_index=block_topm_index[:upper_m],
       k=k,
     ),
     lambda *args: args,
     topk_logits,
     topk_flat_indices,
   )
 # the packed sort is only valid if all topk are >=0 because we do a i32
 # sort with the bf16 bits packed which sorts negative values in reverse order
 # if this wasnt true we fallback to a classic slow topk
 # TODO: revert to f32 and i32 block_topk rather than packed version
 '''
 topk_logits, topk_flat_indices = jax.lax.cond(
   (topk_logits[:,-1] >= 0).all(),
   lambda *args: args,
   lambda *args: tuple(jax.lax.top_k(logits, k)),
   (topk_logits, topk_flat_indices)
 )
 '''
 return topk_logits, topk_flat_indices



In [28]:
topk_optimized(logits, 64, m_stage1_schedule=())

(16, 128) (16, 201088) int32 int32
[63 63 63 63 63 63 63 63 63 63 63 63 63 63 63 63 63 63 63 63 63 63 63 63
 63 63 63 63 63 63 63 63] [Array([[1.02344, 0.546875, -inf, ..., 0.259766, -inf, 0.835938],
       [-inf, 0.558594, -inf, ..., -inf, -inf, 0.691406],
       [0.613281, -inf, -inf, ..., 0.036377, -inf, -inf],
       ...,
       [0.578125, 1.69531, 0.425781, ..., 0.40625, -inf, 1.14844],
       [-inf, -inf, -inf, ..., 0.609375, -inf, -inf],
       [-inf, 0.237305, 0.628906, ..., 1.30469, 0.275391, -inf]],      dtype=bfloat16)] [Array([[ 1570,  1570, 65535, ...,  1570, 65535,  1570],
       [65535,  1570, 65535, ..., 65535, 65535,  1570],
       [ 1570, 65535, 65535, ...,  1570, 65535, 65535],
       ...,
       [ 1570,  1570,  1570, ...,  1570, 65535,  1570],
       [65535, 65535, 65535, ...,  1570, 65535, 65535],
       [65535,  1570,  1570, ...,  1570,  1570, 65535]], dtype=uint16)]


(Array([[2.17188, 2.125, 1.91406, ..., 0.136719, 0.132812, 0.123535],
        [2.98438, 2.5625, 2.45312, ..., 0.253906, 0.249023, 0.237305],
        [2.89062, 2.14062, 1.71875, ..., 0.0361328, 0.0115967, 0.00494385],
        ...,
        [3.20312, 2.85938, 2.70312, ..., 0.223633, 0.199219, 0.176758],
        [2.75, 1.85938, 1.78125, ..., 0.026123, 0.0115356, -inf],
        [1.95312, 1.9375, 1.89062, ..., 0.0201416, 0.00921631, 0.00169373]],      dtype=bfloat16),
 Array([[ 201076,  200975,  201004, ...,  201075,  201003,  200991],
        [ 201084,  200967,  201021, ...,  201073,  200979,  201066],
        [ 201036,  201009,  200965, ...,  200970,  201070,  201026],
        ...,
        [ 201010,  201038,  200980, ...,  201059,  200998,  201029],
        [ 201006,  201054,  201004, ...,  201079,  201019, 8388480],
        [ 201068,  201032,  201057, ...,  201080,  201075,  200966]],      dtype=int32))

In [26]:

k = 64
num_queries = 128
vocab_size = 201088

# To get large value range, randint across uint16, then bitcast to bfloat16, then remove non-normal values
dtype = jnp.uint16
logits = jax.lax.bitcast_convert_type(jax.random.randint(jax.random.key(7), (num_queries, vocab_size), dtype=dtype, minval=jnp.iinfo(dtype).min, maxval=jnp.iinfo(dtype).max), jnp.bfloat16)
logits = jnp.where(jnp.isnan(logits) | (logits==jnp.inf), 0, logits) # remove the nans and +infs

logits = jax.random.normal(jax.random.key(0), logits.shape).astype(jnp.bfloat16)

logits_2 = jnp.ones_like(logits)
# Adversarial logits, in practice this is astronomically unlikely
logits_worst_case = jnp.zeros((num_queries, vocab_size)).at[...,::128].set(1.)

all_values_match = (jax.lax.top_k(logits, k)[0] == topk_optimized(logits, k=k)[0]).all()
exact_index_match =  (jax.lax.top_k(logits, k)[1] == topk_optimized(logits, k=k)[1]).mean()
print(f'''All topk_logits match = {all_values_match}. Indices match at {exact_index_match:.0%} of the time,
this is only O(40%) [not 100%] as bf16 has only 2**16=65k possible values so in 200k vocab size
theres high degeneracy and sorting is different. Having checked, it looks correct. Writing full checks would be tricky.''')


[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4] [Array([[1.17969, -inf, -inf, ..., -inf, -inf, -inf],
       [1.67969, -inf, -inf, ..., 1.16406, -inf, -inf],
       [0.0600586, 0.574219, -inf, ..., 0.414062, 0.0810547, 1.15625],
       ...,
       [-inf, 0.178711, 1.07031, ..., 0.890625, -inf, -inf],
       [0.746094, -inf, -inf, ..., -inf, 0.390625, 0.496094],
       [-inf, 0.349609, -inf, ..., 0.550781, -inf, 0.170898]],      dtype=bfloat16)] [Array([[ 1570, 65535, 65535, ..., 65535, 65535, 65535],
       [ 1570, 65535, 65535, ...,  1570, 65535, 65535],
       [ 1570,  1570, 65535, ...,  1570,  1570,  1570],
       ...,
       [65535,  1570,  1570, ...,  1570, 65535, 65535],
       [ 1570, 65535, 65535, ..., 65535,  1570,  1570],
       [65535,  1570, 65535, ...,  1570, 6553

In [27]:
import functools
import jax
import jax.numpy as jnp

k = 64
num_queries = 32
vocab_size = 201088
hidden_dim = 2880

logit_key, key_act, key_weight = jax.random.split(jax.random.key(0), 3)
x = jax.random.normal(key_act, (num_queries, hidden_dim), dtype=jnp.bfloat16)
w = jax.random.normal(key_weight, (hidden_dim, vocab_size), dtype=jnp.bfloat16)
logits = jax.random.normal(key_weight, (num_queries, vocab_size), dtype=jnp.float32).astype(jnp.bfloat16)

topk_xla = jax.jit(jax.lax.top_k, static_argnames=('k',))
approx_topk_xla = jax.jit(jax.lax.approx_max_k, static_argnames=('k',))

@jax.jit
@functools.partial(jax.vmap, in_axes=(0, None))
def matmul_and_topk_xla(x, w, k=k):
  logits = (x @ w)
  return jax.lax.top_k(logits, k)

def run():
  # reference runtimes
  o = jax.block_until_ready(x @ w)
  jax.block_until_ready(matmul_and_topk_xla(x, w))
  jax.block_until_ready(topk_xla(logits, k=k))

  # Optimized tpu run on random logits
  jax.block_until_ready(topk_optimized(logits, k=k))

  # Optimized tpu on adversarial logits, to check astronomically unlikely worst case runtime
  jax.block_until_ready(topk_optimized(logits_worst_case, k=k))

  # Not exact. Runtime varies with recall, here run with default 0.95
  jax.block_until_ready(approx_topk_xla(logits, k=k))



run()
with jax.profiler.trace("/tmp/tensorboard"):
  run()


(16, 128) (16, 201088) int32 int32
(16, 128) (16, 201088) int32 int32
(16, 128) (16, 201088) int32 int32
(16, 128) (16, 201088) int32 int32
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4] [Array([[1.02344, 0.546875, -inf, ..., 0.259766, -inf, 0.835938],
       [-inf, 0.558594, -inf, ..., -inf, -inf, 0.691406],
       [0.613281, -inf, -inf, ..., 0.036377, -inf, -inf],
       ...,
       [0.578125, 1.69531, 0.425781, ..., 0.40625, -inf, 1.14844],
       [-inf, -inf, -inf, ..., 0.609375, -inf, -inf],
       [-inf, 0.237305, 0.628906, ..., 1.30469, 0.275391, -inf]],      dtype=bfloat16)] [Array([[ 1570,  1570, 65535, ...,  1570, 65535,  1570],
       [65535,  1570, 65535, ..., 65535, 65535,  1570],
       [ 1570, 65535, 65535, ...,  1570, 65535, 65535],
       ...,
       [ 1570,  1570,  1570, ...,  1570, 65535,  1570],
       [65535, 65535, 65535, ...,  1570, 65535, 65535],
       [65535,  1570,  1570, ...,  1570,  1570, 65535]], dtype=uint16)]


AssertionError: 

In [ ]:
# 32 tokens, top-64, v5e: topk_xla 1.32ms, topk_pallas_tpu 0.120ms (11x faster)
# 2048 tokens, top-64, v5e: topk_xla 87.25ms, topk_pallas_tpu 7.23ms (12x faster)


In [ ]:
jnp.array(6, jnp.int32) << 16

Array(393216, dtype=int32)